In [40]:
import pandas as pd

In [41]:
# # Step 1: Read the original CSV
# df = pd.read_csv('Forestry_Trade_Flows_All_Data/Forestry_Trade_Flows_E_All_Data.csv', encoding='ISO-8859-1')
# print(len(df))
# # Step 2: Split the DataFrame into two halves
# split_index = len(df) // 2
# df1 = df.iloc[:split_index]
# df2 = df.iloc[split_index:]

# # Step 3: Save each part to a separate CSV file
# df1.to_csv('Forestry_Trade_Flows_All_Data/Forestry_Trade_Flows_E_All_Data_1.csv', index=False)
# df2.to_csv('Forestry_Trade_Flows_All_Data/Forestry_Trade_Flows_E_All_Data_2.csv', index=False)

# Step 4: Read the two CSV files


In [46]:
def current_network_study():
    df1 = pd.read_csv('Forestry_Trade_Flows_All_Data/Forestry_Trade_Flows_E_All_Data_1.csv', encoding='ISO-8859-1')
    df2 = pd.read_csv('Forestry_Trade_Flows_All_Data/Forestry_Trade_Flows_E_All_Data_2.csv', encoding='ISO-8859-1')
    # Step 5: Concatenate the two DataFrames
    df = pd.read_csv('Forestry_Trade_Flows_All_Data/Forestry_Trade_Flows_E_All_Data.csv', encoding='ISO-8859-1')
    #df = pd.concat([df1, df2])
    # >10% possible links
    df.rename(columns={'Reporter Countries': 'Source', 'Partner Countries': 'Target'}, inplace=True)
    
    # Get nodes
    country_df = df.drop_duplicates(subset=['Source'])
    dict = {item: count for count, item in enumerate(country_df['Source'])}
    country_df['Id'] = df['Source'].apply(lambda x: dict.get(x, -1))
    country_df[['Id', 'Source']].to_csv('current_network_study/countries.csv', index=False)

    # Filter for Year 2018 and non-null values
    filtered_df = df[(df['Item'] == 'Forest products (export/import)') & (df['Element'] == 'Export Value') & (df['Item'] == 'Forest products (export/import)') & (df['Target'] != 'Total FAO') & (df['Target'] != 'Others (adjustment)') & (df['Target'] != 'Unspecified Area') & (df['Y2018'].notna())]
    #filtered_df['Y2018'] = pd.to_numeric(filtered_df['Y2018'], errors='coerce')
    sorted_df = filtered_df.sort_values(by='Y2018', ascending=False)
    sorted_df = sorted_df[['Source', 'Target']]
    sorted_df['Source'] = sorted_df['Source'].replace(dict)
    sorted_df['Target'] = sorted_df['Target'].replace(dict)

    maximum_edge = len(df['Source'].unique())
    maximum_num_edge = maximum_edge * (maximum_edge-1)
    density = len(filtered_df) / maximum_num_edge

    # Different Thresholds
    print(len(sorted_df))
    fifteen_percent = int(0.15*maximum_num_edge)
    ten_percent = int(0.1*maximum_num_edge)
    five_percent = int(0.05*maximum_num_edge)

    fifteen_df = sorted_df.head(fifteen_percent)
    ten_df = sorted_df.head(ten_percent)
    five_df = sorted_df.head(five_percent)
    print(density, fifteen_percent, ten_percent, five_percent)
    
    fifteen_df.to_csv('current_network_study/fifteen_percent.csv', index=False)
    ten_df.to_csv('current_network_study/ten_percent.csv', index=False)
    five_df.to_csv('current_network_study/five_percent.csv', index=False)
    
    # Total FAO
current_network_study()

8380
0.28162387417663665 4463 2975 1487


/var/folders/h5/5nz0lxy96xxbngwn5250d00h0000gn/T/ipykernel_2986/2050910066.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_df['Id'] = df['Source'].apply(lambda x: dict.get(x, -1))
/var/folders/h5/5nz0lxy96xxbngwn5250d00h0000gn/T/ipykernel_2986/2050910066.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sorted_df['Source'] = sorted_df['Source'].replace(dict)
/var/folders/h5/5nz0lxy96xxbngwn5250d00h0000gn/T/ipykernel_2986/2050910066.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated an

In [ ]:
df = pd.read_csv('Forestry_Trade_Flows_All_Data/Forestry_Trade_Flows_E_All_Data.csv', encoding='ISO-8859-1')
filtered_df1 = df[(df['Item'] != 'Forest products (export/import)')]
print(filtered_df1['Y2018'].sum())
filtered_df2 = df[(df['Item'] == 'Forest products (export/import)')]
print(filtered_df2['Y2018'].sum())

3431812903.0
1101411872.0
